In [1]:
import pandas as pd
import json
def prettyjson(final_dict):
    print(json.dumps(final_dict, ensure_ascii=False, indent=4))

df_nurse_record = pd.read_csv("data_processing/NurseRecord_full.csv")

/var/folders/pj/6yc3mtz11t5bfkf7bljn52z40000gn/T/ipykernel_11597/2924052718.py:6: DtypeWarning: Columns (9,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df_nurse_record = pd.read_csv("data_processing/NurseRecord_full.csv")


In [2]:
def get_nursing_record(patient_id: int):
    patient_data = df_nurse_record[df_nurse_record['연구등록번호'] == patient_id]
    columns = ['간호중재_코드명', '간호활동_코드명', '간호속성코드_코드명', '속성', 'Duty_코드명', '시행일시']
    result = patient_data[columns].sort_values('시행일시')

    # 시행일시별로 그룹화
    final_dict = {}
    grouped = result.groupby('시행일시')

    for time_key, group in grouped:
        # 각 그룹의 데이터를 리스트로 변환
        time_data = []
        for index, row in group.iterrows():
            time_data.append({
                "간호중재(코드명)": row['간호중재_코드명'],
                "간호활동(코드명)": row['간호활동_코드명'], 
                "간호속성코드(코드명)": row['간호속성코드_코드명'],
                "속성": row['속성'],
                "Duty(코드명)": row['Duty_코드명'],
                "시행일시": row['시행일시']
            })
        
        final_dict[time_key] = time_data
    return final_dict

In [3]:
# 실제 입퇴원 기간을 몰라. 그러니 해당 기간에 처음 울린 알람과 나중에 울린 알람
def get_admission_periods(patient_id):
    r = json.loads(open(f'TestData/{patient_id}.json', 'r', encoding='utf-8').read())
    admission_periods = []
    for idx, i in enumerate(sorted(list(r['DatSeq'].keys()), key=int)):
        alarms = list(r['DatSeq'][i].keys())
        admission_periods.append({
            "start": alarms[0].split(' ')[0],
            "end": alarms[-1].split(' ')[0],
            "id": f"admission{idx+1}"
        })
    return admission_periods

In [ ]:
# TimeStamp
# PatSeq
# SequenceNumber
# AlertId
# Source
# Label
# Severity
# Kind
# AnnounceTime
# AlertTime
# II_wave
# II_time_diff_sec
# ABP_wave
# ABP_time_diff_sec
# Pleth_wave
# Pleth_time_diff_sec
# Resp_wave
# Resp_time_diff_sec
# SpO2_numeric
# SpO2_numeric_time_diff_sec
# Pulse_numeric
# Pulse_numeric_time_diff_sec
# ST_numeric
# ST_time_diff_sec
# Tskin_numeric
# Tskin_numeric_time_diff_sec
# ABP_numeric
# ABP_numeric_time_diff_sec
# NBP_numeric
# NBP_numeric_time_diff_sec
# Perf_numeric
# Perf_numeric_time_diff_sec
# PPV_numeric
# PPV_numeric_time_diff_sec
# AlertRow
# ClosestNursing
# Nursing_TimeDiffSec

In [4]:
def get_alarms_and_waveforms(patient_id):
    r = json.loads(open(f'TestData/{patient_id}.json', 'r', encoding='utf-8').read())
    waveforms = {}
    alarms = {}

    for idx, i in enumerate(sorted(list(r['DatSeq'].keys()), key=int)):
        for ts in list(r['DatSeq'][i].keys()):

            
            # 2024-01-09 15:32:20.657
            data = r['DatSeq'][i][ts][0]

            # 2024-01-09
            date = ts.split(' ')[0]

            # 15:32:20
            hms = ts.split(' ')[-1].split('.')[0]
            
            
            
            Severity = data["Severity"][0]
            Severity_color = {0: "Red", 1:"Yellow", 2:"ShortYellow", 3:"SevereCyan", 4:"Cyan", 5:"SilentCyan", 6:"White"}[Severity]

            II_time_diff_sec = data["II_time_diff_sec"]
            II_wave = data["II_wave"]["WaveSamples_Base64_cal"]

            ABP_time_diff_sec = data["ABP_time_diff_sec"]
            ABP_wave = data["ABP_wave"]["WaveSamples_Base64_cal"]

            Resp_time_diff_sec = data["Resp_time_diff_sec"]
            Resp_wave = data["Resp_wave"]["WaveSamples_Base64_cal"]

            Pleth_time_diff_sec = data["Pleth_time_diff_sec"]
            Pleth_wave = data["Pleth_wave"]["WaveSamples_Base64_cal"]


            if not f'admission{idx+1}' in alarms.keys():
                alarms[f'admission{idx+1}'] = {}
            if not date in alarms[f'admission{idx+1}'].keys():
                alarms[f'admission{idx+1}'][date] = []


            # 이거 하기 전에 조건 거는걸로 하자.
            alarms[f'admission{idx+1}'][date].append({
                "time": hms,
                "color": Severity_color,
                "id": f"{patient_id}-{date}-{hms}",
                "timestamp": f"{date} {hms}"
            })
            # 이거 하기 전에 조건 거는걸로 하자.
            waveforms[f"{date} {hms}"] = {
                "ABP": ABP_wave,
                "Lead-II": II_wave,
                "Resp": Resp_wave,
                "Pleth": Pleth_wave
            }
            # break # 하나만 해서 넣고싶을 때 주석 풀면 됨.

    return alarms, waveforms
                


In [5]:
patient_id = 10216420
admissions = get_admission_periods(patient_id)
nursing_records = get_nursing_record(patient_id)
alarms, waveforms = get_alarms_and_waveforms(patient_id)


DataConverter = {
    "admission_periods": admissions,
    "alarms": alarms,
    "nursing_records": nursing_records,
    "waveforms": waveforms
}

if not nursing_records:
    print(patient_id)

with open(f'DATA/{patient_id}.json', 'w', encoding='utf-8') as f:
    json.dump(DataConverter, f, ensure_ascii=False, indent=4)

print(f"DATA/{patient_id}.json 파일 저장 완료!")

# import os
# ids = [i[:-5] for i in os.listdir('TestData')]

DATA/10216420.json 파일 저장 완료!
